# Problem 1: SQL Operations

This problem will test your ability to manipulate two simple SQL tables. You may find a problem easier to complete using Pandas, or you may find a problem easier to complete in SQL. We will provide you will a SQLite database containing two tables, and two Pandas Dataframes that are identical to the SQLite tables. 

Recall that on the current version of the Vocareum platform, the `sqlite3` module only works with their Python 3.5 build, rather than the 3.6 build we usually use. The cell below imports the necessary modules and prints their versions. As such, **if** you are prototyping on your local machine with a version of Python greater than 3.5 or using versions of the pandas and SQLite modules that differ from what is on Vocareum, you may need to make adjustments to pass the autograder. **Doing so is your responsibility so budget your time accordingly.**

In [1]:
import sys
import pandas as pd
import sqlite3 as db
from IPython.display import display

print("* Python version: {}".format(sys.version))
print("* pandas version: {}".format(pd.__version__))
print("* sqlite3 version: {}".format(db.version))

* Python version: 3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]
* pandas version: 0.23.4
* sqlite3 version: 2.6.0


## The Movies and Cast Dataset

The data consists of two tables. The first is a table of movies along with (random) audience scores from 1-100. The second is a table of cast members for those movies. There are some interesting cast members in here that you might stumble upon!

Let's read in the database file and show the table descriptions.

In [2]:
disk_engine = db.connect('movieDB.db')
c = disk_engine.cursor()

c.execute('SELECT type, name, sql FROM sqlite_master')
results = c.fetchall()
for table in results:
    print(table)

('table', 'movies', 'CREATE TABLE movies (id integer, name text, score integer)')
('table', 'cast', 'CREATE TABLE cast (movie_id integer, cast_id integer, cast_name text)')


In [3]:
movies = pd.read_table('movie-name-score.txt', sep=',', header=None, names=['id', 'name', 'score'])
cast = pd.read_table('movie-cast.txt', sep=',', header=None, names=['movie_id', 'cast_id', 'cast_name'])

print('Movies Dataframe:')
print('-------------------')
display(movies.head())
print('\n\n')
print('Cast Dataframe:')
print('-------------------')
display(cast.head())

Movies Dataframe:
-------------------


,id,name,score
0,9,Star Wars: Episode III - Revenge of the Sith 3D,61
1,24214,"The Chronicles of Narnia: The Lion, The Witch ...",46
2,1789,War of the Worlds,94
3,10009,Star Wars: Episode II - Attack of the Clones 3D,28
4,771238285,Warm Bodies,3





Cast Dataframe:
-------------------


,movie_id,cast_id,cast_name
0,9,162652153,Hayden Christensen
1,9,162652152,Ewan McGregor
2,9,418638213,Kenny Baker
3,9,548155708,Graeme Blundell
4,9,358317901,Jeremy Bulloch


In terms of Database structures, the **`cast`** table's **`movie_id`** column is a foreign key to the **`movie`** table's **`id`** column. 

This means you can perform any SQL joins or Pandas merges between the two tables on this column. 

One final code cell to get you started - implement the all-too-familiar `canonicalize_tibble` and `tibbles_are_equivalent` functions. 

In [4]:
def canonicalize_tibble(X):
    var_names = sorted(X.columns)
    Y = X[var_names].copy()
    Y.sort_values(by=var_names, inplace=True)
    Y.reset_index(drop=True, inplace=True)
    return Y

def tibbles_are_equivalent (A, B):
    A_canonical = canonicalize_tibble(A)
    B_canonical = canonicalize_tibble(B)
    equal = (A_canonical == B_canonical)
    return equal.all().all()

Let's start with two warm-up exercises. 

**Exercise 0** (2 points): Create a dataframe, ***`cast_size`***, that contains the number of distinct cast members per movie. Your table will have two columns, *`movie_name`*, the name of each film, and *`cast_count`*, the number of unique cast members for the film. 

Order the result by *`cast_count`* from highest to lowest.

In [5]:
# https://stackoverflow.com/questions/15411158/pandas-countdistinct-equivalent

#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html

cast_id=cast.groupby('movie_id').cast_name.nunique().to_frame()
#cast_size.columns=['movie_name', 'cast_count']
cast_id.reset_index(inplace=True)


cast_size=cast_id.merge(movies, left_on='movie_id', right_on='id')
cast_size=cast_size[['name','cast_name']]
cast_size.columns=['movie_name','cast_count']
cast_size.sort_values(by='cast_count', ascending=False,inplace=True)
cast_size.reset_index(inplace=True, drop=True)
print(cast_size)

                                            movie_name  cast_count
0                                The War of the Worlds          72
1      Star Wars: Episode III - Revenge of the Sith 3D          66
2    The Chronicles of Narnia: The Lion, The Witch ...          66
3           Star Wars: Episode VI - Return of the Jedi          65
4                                       This Means War          64
5                                              Warrior          62
6                                           Hart's War          59
7                                        War and Peace          58
8                                         The Warriors          57
9                                      In Love and War          56
10                                         Hanna's War          56
11                                           War Horse          53
12                                           War, Inc.          50
13                               Oh! What a Lovely War        

In [6]:
# Test cell : `test_cast_size`

print("Reading instructor's solution...")

cast_size_solution = pd.read_csv('../resource/asnlib/publicdata/cast_size_solution.csv')

print("Checking...")

assert set(cast_size.columns) == {'movie_name', 'cast_count'}
assert tibbles_are_equivalent(cast_size, cast_size_solution), "Your Dataframe is incorrect"
assert all(cast_size['cast_count'] == cast_size_solution['cast_count'])


print("\n(Passed!.)")

del cast_size_solution

Reading instructor's solution...
Checking...

(Passed!.)


**Exercise 1** (2 point): Create a dataframe, **`cast_score`**, that contains the average movie score for each cast member. Your table will have two columns, *`cast_name`*, the name of each cast member, and *`avg_score`*, the average movie review score for each movie that the cast member appears in. 

Order this result by `avg_score` from highest to lowest, and round your result for `avg_score` to two (2) decimal places. 

Break any ties in your sorting by cast name in alphabetical order from A-Z. 

In [7]:
# https://stackoverflow.com/questions/17141558/how-to-sort-a-dataframe-in-python-pandas-by-two-or-more-columns

big_df=cast.merge(movies, left_on='movie_id', right_on='id')
cast_score=big_df.groupby('cast_name').score.mean().round(2).to_frame()
cast_score.reset_index(inplace=True)
cast_score.columns=['cast_name','avg_score']
cast_score.sort_values(['avg_score','cast_name'],inplace=True,ascending=[False,True])

cast_score.reset_index(inplace=True,drop=True)
#display(cast_score)


In [8]:
# Test cell : `test_cast_score`
print("Reading instructor's solution...")

cast_score_solution = pd.read_csv('../resource/asnlib/publicdata/cast_score_solution.csv')

print("Checking...")

assert set(cast_score.columns) == {'cast_name', 'avg_score'}
assert tibbles_are_equivalent(cast_score, cast_score_solution), "Your Dataframe is incorrect"
assert all(cast_score['avg_score'] == cast_score_solution['avg_score'])


print("\n(Passed!)")

del cast_score_solution



Reading instructor's solution...
Checking...

(Passed!)


**Exercise 2** (3 points): You will now create a dataframe, **`one_hit_wonders`**, that contains actors and actresses that appear in **exactly** one movie, with a movie score == 100. Your result will have three columns, *`cast_name`,* the name of each cast member that meets the criteria, *`movie_name`*, the name of the movie that cast member appears in, and *`movie_score`*, which for the purposes of this Exercise is always == 100. 

Order your result by `cast_name` in alphabetical order from A-Z.

In [9]:
#display(big_df)

In [10]:
#Create dataframe of score=100 movies
score100=big_df[big_df.score==100]

#Create dataframe of actors appearing in one movie
actor_movie=big_df.groupby(['cast_name'])['movie_id'].nunique().to_frame()

actor_movie.reset_index(inplace=True)
actor_movie_one=actor_movie[actor_movie.movie_id==1]
actor_movie_one.rename({'movie_id':'one_film'},axis=1,inplace=True)

#display(actor_movie_one)


#Get df with relevant info - actors
df=actor_movie_one.merge(big_df,left_on='cast_name',right_on='cast_name',how='inner')

df2=df[['cast_name','name','score']]
df3=df2[df2.score==100]
df3.rename({'name':'movie_name','score':'movie_score'},axis=1,inplace=True)
df3.reset_index(inplace=True,drop=True)
display(df3)
one_hit_wonders=df3


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,cast_name,movie_name,movie_score
0,Alison Lohman,Kaze no tani no Naushika (Nausicaa of the Vall...,100
1,Andrew Lauer,Hg Wells War of the Worlds,100
2,Chris Sarandon,Kaze no tani no Naushika (Nausicaa of the Vall...,100
3,Clare Holman,Island at War,100
4,Jake Busey,Hg Wells War of the Worlds,100
5,James Wilby,Island at War,100
6,Joanne Froggatt,Island at War,100
7,Julia Ford,Island at War,100
8,Patrick Stewart,Kaze no tani no Naushika (Nausicaa of the Vall...,100
9,Peter Greene,Hg Wells War of the Worlds,100


In [11]:
# Test cell : `one_hit_wonders_score`

print("Reading instructor's solution...")

one_hit_wonders_solution = pd.read_csv('../resource/asnlib/publicdata/one_hit_wonders_solution.csv')

print("Checking...")

assert set(one_hit_wonders.columns) == {'cast_name','movie_name', 'movie_score'}
assert tibbles_are_equivalent(one_hit_wonders, one_hit_wonders_solution)
assert all(one_hit_wonders['movie_score'] == one_hit_wonders_solution['movie_score'])

print("\n(Passed!)")

del one_hit_wonders_solution

Reading instructor's solution...
Checking...

(Passed!)


In [12]:
#display(one_hit_wonders_solution)

**Exercise 3** (3 points): For this problem, you will find cast members that work well together. We define this as two cast members being in **>= 3** movies together, with the **average movie score being >= 50**. 

You will create a dataframe called **`good_teamwork`** that contains four columns:
- *`cast_member_1`* and *`cast_member_2`*, the names of each pair of cast members that appear in the same movie;
- *`num_movies`*, the number of movies that each pair of cast members appears in; and
- *`avg_score`*, the average review score for each of those movies containing the two cast members. 

Order the results by `cast_member_1` alphabetically from A-Z, and break any ties by sorting by `cast_member_2` alphabetically from A-Z. Round the result for `avg_score` to two (2) decimal places.

One more wrinkle: your solution will likely create several duplicate pairs of cast members: rows such as:

cast_member_1     |cast_member_2  |num_movies  |avg_score
------------------|---------------|------------|---------
 Anthony Daniels  |Frank Oz       |5           |50.60
 Frank Oz         |Anthony Daniels|5           |50.60
 
Remove all duplicate pairs, keeping all cases where `cast_member_1`'s name comes before `cast_member_2`'s name in the alphabet. In the example above, you will keep **only** the first row in your final solution. Make sure to also remove matches where `cast_member_1` == `cast_member_2`.

In [13]:
#display(big_df)

import numpy as np

# https://stackoverflow.com/questions/42375225/pythonic-way-to-create-pairs-of-values-in-a-column-in-dataframe
df=big_df.copy()
df.rename({'cast_name':'cast_member_','score':'movie_score'},axis=1,inplace=True)
df1=pd.merge(df, df, on='movie_id', suffixes=['1','2']).query("cast_member_1 != cast_member_2")
#display(df1)

#Count the number of movies for each pair
counts=df1.groupby(['cast_member_1','cast_member_2']).size().to_frame()
counts.columns=['num_movies']

#Average movie score for each pair
scores=df1.groupby(['cast_member_1','cast_member_2']).movie_score2.mean().to_frame()
scores.rename({'movie_score2':'avg_score'},axis=1,inplace=True)

#Keep only scores >= 50
scores=scores[scores.avg_score >= 50.0]
#print(len(scores))

#display(scores)
#Enforce condition that pair must appear in >=3 movies together
counts=counts[counts['num_movies']>=3]
#print(len(counts))

#desired_df=counts.merge


desired_df=pd.concat([counts, scores], axis=1)
desired_df.dropna(inplace=True)

desired_df.reset_index(inplace=True)
#desired_df.drop_duplicates(keep=False, inplace=True)

#desired_df = desired_df[~desired_df.duplicated(subset=['cast_member_1','cast_member_2'],keep='first')]

#display(desired_df)

#Remove mirror pairs from cast_member_1 and cast_member_2 columns
# https://stackoverflow.com/questions/48549637/pandas-removing-mirror-pairs-from-dataframe

good_teamwork=desired_df.loc[pd.DataFrame(np.sort(desired_df[['cast_member_1','cast_member_2']],1),index=desired_df.index).drop_duplicates(keep='first').index]
good_teamwork.reset_index(drop=True,inplace=True)

good_teamwork.num_movies=good_teamwork.num_movies.astype(int)
good_teamwork.avg_score=round(good_teamwork.avg_score,2)

display(good_teamwork)

#good_teamwork= pd.read_csv('../resource/asnlib/publicdata/good_teamwork_solution.csv')

,cast_member_1,cast_member_2,num_movies,avg_score
0,Ahmed Best,Anthony Daniels,3,54.67
1,Ahmed Best,Ewan McGregor,3,54.67
2,Ahmed Best,Frank Oz,3,54.67
3,Ahmed Best,Ian McDiarmid,3,54.67
4,Ahmed Best,Kenny Baker,3,54.67
5,Ahmed Best,Natalie Portman,3,54.67
6,Ahmed Best,Oliver Ford Davies,3,54.67
7,Ahmed Best,Samuel L. Jackson,3,54.67
8,Ahmed Best,Silas Carson,3,54.67
9,Anthony Daniels,Ewan McGregor,3,54.67


In [14]:
# Test cell : `good_teamwork_score`
print("Reading instructor's solution...")

good_teamwork_solution = pd.read_csv('../resource/asnlib/publicdata/good_teamwork_solution.csv')
print(good_teamwork_solution)

print("Checking...")

assert set(good_teamwork.columns) == {'cast_member_1','cast_member_2', 'num_movies', 'avg_score'}
assert tibbles_are_equivalent(good_teamwork, good_teamwork_solution)
assert all(good_teamwork['num_movies'] == good_teamwork_solution['num_movies'])
assert all(good_teamwork['avg_score'] == good_teamwork_solution['avg_score'])

print("\n(Passed!)")

del good_teamwork_solution

Reading instructor's solution...
         cast_member_1       cast_member_2  num_movies  avg_score
0           Ahmed Best     Anthony Daniels           3      54.67
1           Ahmed Best       Ewan McGregor           3      54.67
2           Ahmed Best            Frank Oz           3      54.67
3           Ahmed Best       Ian McDiarmid           3      54.67
4           Ahmed Best         Kenny Baker           3      54.67
5           Ahmed Best     Natalie Portman           3      54.67
6           Ahmed Best  Oliver Ford Davies           3      54.67
7           Ahmed Best   Samuel L. Jackson           3      54.67
8           Ahmed Best        Silas Carson           3      54.67
9      Anthony Daniels       Ewan McGregor           3      54.67
10     Anthony Daniels            Frank Oz           5      50.60
11     Anthony Daniels       Ian McDiarmid           4      53.75
12     Anthony Daniels      Jeremy Bulloch           3      50.00
13     Anthony Daniels     Natalie Portman 

In [15]:
c.close()
disk_engine.close()

**Fin!** Remember to test your solutions by running them as the autograder will: restart the kernel and run all cells from "top-to-bottom." Also remember to submit to the autograder; otherwise, you will **not** get credit for your hard work!